In [1]:
import os
import time
import configparser

config = configparser.ConfigParser()
config.read('config_user.ini')

#TO BE CONFIGURED FROM CONFIG FILE
os.environ['ICTDIR'] = config['Paths']['ICTDIR']
os.environ["PATH"] = os.environ["PATH"] + config['Paths']['PATH_APPEND']
# Define the folder base path these are to run in local
base_path = config['Paths']['BASE_PATH']
path_to_wf = config['Paths']['PATH_TO_WF']

print(os.environ["PATH"])

/Users/samuele/Software/miniforge3/envs/ic/bin:/Users/samuele/Software/miniforge3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/Users/samuele/Software/IC/bin:/Users/samuele/Software/miniforge3/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/Users/samuele/Software/IC/bin/


In [2]:
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import numpy as np
from scipy.signal import find_peaks, peak_widths

In [3]:
from ipywidgets import widgets, VBox, HBox
from ipywidgets import interactive
from IPython.display import display
from IPython.display import display, HTML

In [4]:
#MUST BE RUNNED WITH IC ENV
from invisible_cities.cities.components import deconv_pmt 
from invisible_cities.database.load_db import DataSiPM
from invisible_cities.database.load_db import DataPMT

In [5]:
display(HTML("""
<style>
.container {
    width: 95% !important;  /* Adjust the percentage as needed */
    max-width: 80%;        /* Optional: prevents exceeding the screen width */
    margin-left: auto;      /* Centers the content */
    margin-right: auto;     /* Centers the content */
}
</style>
"""))

import warnings
# Suppress all RuntimeWarnings
warnings.simplefilter("ignore", RuntimeWarning)

#Configfile path must not be changed
config_file_path = "/tmp/temp_irene.conf"

### Run Sel buttons

In [6]:
# Input widget for the run number
run_number_input_form = widgets.Text(
    description='Run Number:',
    placeholder='Enter run number',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='230px')
)

# Button to check folder existence
check_run_button = widgets.Button(
    description='Check Run',
    button_style='info'
)

# Output area to display the result of run selection
output = widgets.Output()

### Event sel buttons

In [7]:
# Slider for selecting a number
event_select_slider= widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    description='Event Number:',
    style={'description_width': 'initial'},
    disabled=True
)

# Button to plot the PMT waveforms
event_selection_button = widgets.Button(
    description='Select event',
    button_style='success',
    style={'description_width': 'initial'},
    disabled = True,
    layout=widgets.Layout(width='100px')
)

# Create progress bar and output area
progress_bar_load_ev = widgets.IntProgress(
    min=0, 
    max=100, 
    value=50, 
    description="", 
    bar_style="",
    layout=widgets.Layout(width='120px',visibility='hidden')
)

NextEvent_button = widgets.Button(
    description='>>',
    button_style='success',  
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Set the button width to make it smaller
)

PrevEvent_button = widgets.Button(
    description='<<',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Set the button width to make it smaller
)

progress_bar_output = widgets.Output()

### PMT display buttons

In [8]:
# Manual input for PMT number (integer between -1 and pmtrwf.shape[0])
pmt_number_input = widgets.BoundedIntText(
    value=0,
    min=-1,
    max=100,  # Placeholder, will update after loading data
    description='PMT Number (-1 for sum):',
    style={'description_width': 'initial'},
    disabled = True,
    layout=widgets.Layout(width='230px')
)

checkbox_logscale_PMTrw = widgets.Checkbox(
    value=False,  # Default value
    description='log/lin',  # Label for the checkbox
    indent=False,  # Removes the default indentation
    layout=widgets.Layout(width='130px')
)

# Manual input for deconv parameter (integer > 0 and < pmtrwf.shape[1])
deconv_par_input = widgets.BoundedIntText(
    value=100,
    min=1,
    max=100,  # Placeholder, will update after loading data
    description='Deconv Par (default 0.9*len(wf)):',
    style={'description_width': 'initial'},
    disabled = True
)

# Button to plot the PMT waveforms
plot_pmt_button = widgets.Button(
    description='Plot PMT wf(s)',
    button_style='success',
    style={'description_width': 'initial'},
    disabled = True,
    layout=widgets.Layout(width='100px')
)

# Button to set default n value
set_Ndef_button = widgets.Button(
    description='Set default N',
    button_style='',  # No predefined style (gray button)
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='100px')  # Set the button width to make it smaller
)

NextPMT_button = widgets.Button(
    description='>>',
    button_style='success',  
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Set the button width to make it smaller
)

PrevPMT_button = widgets.Button(
    description='<<',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Set the button width to make it smaller
)

# Output widgets
output = widgets.Output()
plot_output = widgets.Output()


### SiPM display buttons

In [9]:
# Manual input for SiPM number (integer between -1 and pmtrwf.shape[0])
SiPM_number_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100,  # Placeholder, will update after loading data
    description='SiPM Number:',
    style={'description_width': 'initial'},
    disabled = True,
    layout=widgets.Layout(width='250px')
)

# Button to plot the PMT waveforms
SiPM_plot_button = widgets.Button(
    description='Plot SiPM wf(s)',
    button_style='success',
    style={'description_width': 'initial'},
    disabled = True,
    layout=widgets.Layout(width='100px')
)

NextSiPM_button = widgets.Button(
    description='>>',
    button_style='success',  
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Set the button width to make it smaller
)

PrevSiPM_button = widgets.Button(
    description='<<',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Set the button width to make it smaller
)

### Statistics buttons

In [10]:
# Create PMT_Stat_Plot button
PMT_Stat_Plot = widgets.Button(
    description='PMT Stat',
    button_style='primary',  # 'primary' for a blue button
    layout=widgets.Layout(width='150px', margin='5px'),
    disabled=True
)

# Create SiPM_Stat_Plot button
SiPM_Stat_Plot = widgets.Button(
    description='SiPM Stat',
    button_style='primary',  # 'primary' for a blue button
    layout=widgets.Layout(width='150px', margin='5px'),
    disabled=True
)

### Functions Run Selection

In [11]:
# Update slider and manual input
def update_slider_and_input(nevents_per_file,file_count):
    max_value = (nevents_per_file) * file_count -1
    event_select_slider.max = max_value
    event_select_slider.disabled = False
    event_selection_button.disabled = False

def disable_all():
    event_select_slider.disabled = True
    event_selection_button.disabled = True
    NextEvent_button.disabled=True
    PrevEvent_button.disabled=True
   
    pmt_number_input.disabled=True
    deconv_par_input.disabled=True
    plot_pmt_button.disabled=True
    set_Ndef_button.disabled=True
    
    SiPM_number_input.disabled=True
    SiPM_plot_button.disabled=True
    
    NextPMT_button.disabled=True
    PrevPMT_button.disabled=True
    NextSiPM_button.disabled=True
    PrevSiPM_button.disabled=True
    
    PMT_Stat_Plot.disabled=True
    SiPM_Stat_Plot.disabled=True
    
    Reco_Irene_Button.disabled=True
    
    hide_display_reco_ui()
    
# Check if the folder exists
def check_folder_exists(b):
    output.clear_output()
    plot_output.clear_output()
    disable_all()
    
    with output:
        global run_number
        run_number = run_number_input_form.value.strip()
        global folder_path
        #folder_path = os.path.join(base_path,str(run_number),path_to_wf)
        folder_path=base_path+"/"+str(run_number)+"/"+ path_to_wf

        if os.path.exists(folder_path):
            print('Run exist. ', end='')
            global filelist
            filelist = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            file_count = len(filelist)

            print(f"Nfiles: {file_count}")

            if file_count>0:
                outcomand = os.popen(f'ptdump {folder_path}{filelist[0]} | grep "pmtrwf" | grep -m 7 -o "[0-9][0-9]" | grep -m 1 "[0-9][0-9]"').read()
            else:
                return
            
            global nevents_per_file
            nevents_per_file=int(outcomand)
            
            # Update slider and inputs
            update_slider_and_input(nevents_per_file,file_count)
        else:
            print(f"Run does not exist")
            event_select_slider.disabled = True
            event_selection_button.disabled = True

### Function Event selection

In [12]:
#Event selection, file open and par setting
def select_event(b):
    global f
    global pmtrwf,sipmrwf
    
    disable_display_comand()
    plot_output.clear_output()
    
    # Update PMT and sample input ranges dynamically
    progress_bar_load_ev.layout.visibility = 'visible'
    progress_bar_load_ev.value = 0
    global event_number
    event_number=event_select_slider.value
    
    global local_ev_number
    filenum_inlist = event_number // nevents_per_file
    local_ev_number = event_number % nevents_per_file
    print(filenum_inlist,local_ev_number)
    progress_bar_load_ev.value=20
    
    global filename
    filename = os.path.join(folder_path, filelist[filenum_inlist])

    global pmtrwf_full, sipmrwf_full
    try:
    # Check if `f` is already defined and in globals()
        if 'f' in globals():
            print("f in globals")
            # Check if the filename of the current file matches
            if hasattr(f, 'filename') and f.filename == filename:
                print("same file, doing nothing")
                progress_bar_load_ev.value=40
            else:
                print("not same file, close and reopening")
                f.close()  # Close the currently opened file to avoid resource leaks
                progress_bar_load_ev.value=40
                f = tb.open_file(filename, 'r')
                pmtrwf_full= f.get_node('/RD/pmtrwf')[:]
                sipmrwf_full=f.get_node('/RD/sipmrwf')[:]
        else:
            print("not in global, opening")
            f = tb.open_file(filename, 'r')  # Initialize `f` and open the file
            progress_bar_load_ev.value=40
            pmtrwf_full= f.get_node('/RD/pmtrwf')[:]
            sipmrwf_full=f.get_node('/RD/sipmrwf')[:]
    except NameError:
        print("f is not defined, opening the file from scratch")
        f = tb.open_file(filename, 'r')  # Initialize `f` and open the file
        progress_bar_load_ev.value=40
        pmtrwf_full= f.get_node('/RD/pmtrwf')[:]
        sipmrwf_full=f.get_node('/RD/sipmrwf')[:]
    
    pmtrwf = pmtrwf_full[local_ev_number, :, :]
    progress_bar_load_ev.value=60
    
    sipmrwf = sipmrwf_full[local_ev_number, :, :]
    progress_bar_load_ev.value=80
    
    update_obj_lim()
    eneable_display_comand()
    progress_bar_load_ev.value=100
    time.sleep(0.5)
    progress_bar_load_ev.layout.visibility = 'hidden'
   
def update_obj_lim():
    global num_pmts,num_sipms
    
    deconv_par_input.max = pmtrwf.shape[1]   
    deconv_par_input.value = int(0.9 * pmtrwf.shape[1])   
    num_pmts=pmtrwf.shape[0]
    pmt_number_input.max = num_pmts - 1
    num_sipms = sipmrwf.shape[0]
    SiPM_number_input.max=num_sipms-1 

def eneable_display_comand():
    NextEvent_button.disabled=False
    PrevEvent_button.disabled=False
    
    pmt_number_input.disabled=False
    deconv_par_input.disabled=False
    plot_pmt_button.disabled=False
    set_Ndef_button.disabled=False
    SiPM_number_input.disabled=False
    SiPM_plot_button.disabled=False
    PMT_Stat_Plot.disabled=False
    SiPM_Stat_Plot.disabled=False
    Reco_Irene_Button.disabled=False
    
    PrevPMT_button.disabled=False
    NextPMT_button.disabled=False
    PrevSiPM_button.disabled=False
    NextSiPM_button.disabled=False

def disable_display_comand():
    NextEvent_button.disabled=True
    PrevEvent_button.disabled=True
    
    pmt_number_input.disabled=True
    deconv_par_input.disabled=True
    plot_pmt_button.disabled=True
    set_Ndef_button.disabled=True
    SiPM_number_input.disabled=True
    SiPM_plot_button.disabled=True
    PMT_Stat_Plot.disabled=True
    SiPM_Stat_Plot.disabled=True
    Reco_Irene_Button.disabled=True
    
    PrevPMT_button.disabled=True
    NextPMT_button.disabled=True
    PrevSiPM_button.disabled=True
    NextSiPM_button.disabled=True
    
    hide_display_reco_ui()
    
def set_next_event(b):
    val=event_select_slider.value
    event_select_slider.value=val+1
    
def set_prev_event(b):
    val=event_select_slider.value
    event_select_slider.value=val-1

### PMT display management

In [28]:

def find_signal_max(vec):
    # Find peaks and their properties (including prominence)
    peaks, properties = find_peaks(vec, prominence=10)
    
    # Filter peaks by prominence < 100
    valid_peaks = [(peak, properties['prominences'][i]) for i, peak in enumerate(peaks) if properties['prominences'][i] < 1580]
    
    if not valid_peaks:
        return None  # Return None if no valid peaks are found
    
    # Get the highest peak (with the maximum value)
    highest_peak = max(valid_peaks, key=lambda x: vec[x[0]])
    
    return vec[highest_peak[0]]  # Return the amplitude of the highest peak

# Plot PMT waveforms
def plot_pmt_wfs(b):
    plot_output.clear_output()
    
    with plot_output:
        time = np.linspace(0, pmtrwf.shape[1] * 25e-3, pmtrwf.shape[1])  # Assuming 25 ns sample spacing
        pmt_num = pmt_number_input.value
        deconv_par = deconv_par_input.value
        # Perform deconvolution (assuming `deconv_pmt` is predefined)
        deconv = deconv_pmt("next100", int(run_number), deconv_par)
        cwf = deconv(pmtrwf)

        if pmt_num == -1:
            
            sum_pmtrwf = pmtrwf.sum(axis=0)  # Sum over all PMTs
            sum_cwf = cwf.sum(axis=0)        # Sum over all PMTs
            
            # Create a figure with two subplots: one for the raw sum and one for the calibrated sum
            fig, axs = plt.subplots(1, 2, figsize=(18, 6))

            # Plot the sum of raw waveforms
            axs[0].plot(time, sum_pmtrwf, label=f"Sum of raw PMT WF ev {event_number} run {run_number}", color="blue")
            axs[0].set_title("Sum of PMTs Raw Waveforms")
            axs[0].set_xlabel("Time [μs]")
            axs[0].set_ylabel("Amplitude")
            #axs[0].set_yscale('log')
            axs[0].grid(True)
            axs[0].legend()

            # Plot the sum of calibrated waveforms
            axs[1].plot(time, sum_cwf, label=f"Sum of deconv PMT WF ev {event_number} run {run_number}", color="orange")
            axs[1].set_title("Sum of PMTs Calibrated Waveforms")
            axs[1].set_xlabel("Time [μs]")
            axs[1].set_ylabel("Amplitude")
            if(checkbox_logscale_PMTrw.value):
                axs[1].set_yscale('log')
            else:
                uprange = find_signal_max(sum_cwf)
                if(uprange>0):
                    axs[1].set_ylim([-50-0.05*uprange,uprange+0.5*uprange])
                else:
                    axs[1].set_ylim([-50,4000])
            axs[1].grid(True)
            axs[1].legend()

            # Adjust layout for better spacing
            plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space for the suptitle
            plt.show()

        elif 0 <= pmt_num < pmtrwf.shape[0]:
            
            fig, axs = plt.subplots(1, 2, figsize=(18, 6))
            axs[0].plot(time, pmtrwf[pmt_num], label="Raw WF")
            axs[0].set_title(f"PMT {pmt_num} Raw WF ev {event_number} run {run_number}")
            axs[0].set_xlabel("Time [μs]")
            axs[0].set_ylabel("Amplitude")
            axs[0].legend()
            axs[0].grid(True)

            axs[1].plot(time, cwf[pmt_num], label=f"Deconv WF \n N={deconv_par}", color="orange")
            axs[1].set_title(f"PMT {pmt_num} Deconv WF ev {event_number} run {run_number}")
            axs[1].set_xlabel("Time [μs]")
            axs[1].set_ylabel("Amplitude")
            if(checkbox_logscale_PMTrw.value):
                axs[1].set_yscale('log')
            else:
                uprange = find_signal_max(cwf[pmt_num])
                if(uprange>0):
                    axs[1].set_ylim([-50-0.05*uprange,uprange+0.5*uprange])
                else:
                    axs[1].set_ylim([-50,4000])
            axs[1].legend()
            axs[1].grid(True)

            plt.tight_layout()
            plt.show()
        else:
            print(f"Invalid PMT number: {pmt_num}")
    
    NextPMT_button.disabled=False
    PrevPMT_button.disabled=False
            
def set_N_default(b):
    deconv_par_input.value=int(0.9 * pmtrwf.shape[1])

def plot_next_pmt(b):
    if (pmt_number_input.value>=0) & (pmt_number_input.value<num_pmts):
        pmt_number_input.value+=1
        plot_pmt_wfs(None)
        
def plot_prev_pmt(b):
    print(num_pmts)
    if (pmt_number_input.value>0) & (pmt_number_input.value<=num_pmts):
        pmt_number_input.value-=1
        plot_pmt_wfs(None)

        

### SiPM display management

In [14]:
def plot_sipm_wf(b):
    SiPM=SiPM_number_input.value
    plot_output.clear_output()
    sp_sipm = 1
    t_sipm  = np.arange(sipmrwf.shape[1]) * sp_sipm
    
    with plot_output:
        # Plot the waveform for the chosen SiPM vs time
        plt.figure(figsize=(15, 6),dpi=160)
        plt.plot(t_sipm, sipmrwf[SiPM], color='r',marker=".")
        plt.title(f'SiPM {SiPM} Raw WF ev {event_number} run {run_number}')
        plt.xlabel('Time [μs]')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.show()
    
    NextSiPM_button.disabled=False
    PrevSiPM_button.disabled=False
    
        
def plot_next_sipm(b):
    if (SiPM_number_input.value>=0) & (SiPM_number_input.value<num_sipms):
        SiPM_number_input.value+=1
        plot_sipm_wf(None)
        
def plot_prev_sipm(b):
    if (SiPM_number_input.value>0) & (SiPM_number_input.value<=num_sipms):
        SiPM_number_input.value-=1
        plot_sipm_wf(None)

### Stats Plot Function

In [15]:
def plotPMT_stat(b):
    PMTPos = DataPMT('next100', 0).filter('XY')
    
    # Extract X and Y positions
    X, Y = PMTPos["X"], PMTPos["Y"]
    
    # Perform deconvolution
    deconv_par=int(0.9 * pmtrwf.shape[1])
    deconv = deconv_pmt("next100", int(run_number), deconv_par)
    cwf = deconv(pmtrwf)
    
    # Calculate standard deviation and max values
    STD = [np.std(cwf[i, :300]) for i in range(num_pmts)]
    max_values = [np.max(cwf[i]) for i in range(num_pmts)]
    
    plot_output.clear_output()
    
    # Plot in plot_output widget
    with plot_output:
        plot_output.clear_output()  # Clear previous plots
        fig, axs = plt.subplots(1, 2, figsize=(14, 6),dpi=160)

        # Plot 1: STD values
        scatter1 = axs[0].scatter(X, Y, c=np.log(STD), s=200, cmap='plasma', alpha=0.8, edgecolors='k')
        axs[0].set_title("Baseline RMS vs PMT Positions")
        axs[0].set_xlabel("X [mm]")
        axs[0].set_ylabel("Y [mm]")
        plt.colorbar(scatter1, ax=axs[0], label="log(STD)")

        # Plot 2: Max values
        scatter2 = axs[1].scatter(X, Y, c=max_values, s=200, cmap='plasma', alpha=0.8, edgecolors='k')
        axs[1].set_title("Max Amp vs PMT Positions")
        axs[1].set_xlabel("X [mm]")
        axs[1].set_ylabel("Y [mm]")
        plt.colorbar(scatter2, ax=axs[1], label="Max Value")

        # Adjust layout and display
        plt.tight_layout()
        plt.show()
        
def plotSiPM_stat(b):
    SiPMpos = DataSiPM('next100', 0).filter('XY')
    
    # Extract X and Y positions
    X, Y = SiPMpos["X"], SiPMpos["Y"]
    
    # Calculate standard deviation and max values
    STD = [np.std(sipmrwf[i, :300]) for i in range(num_sipms)]
    max_values = [np.max(sipmrwf[i]) for i in range(num_sipms)]
    
    plot_output.clear_output()
    
    # Plot in plot_output widget
    with plot_output:
        plot_output.clear_output()  # Clear previous plots
        fig, axs = plt.subplots(1, 2, figsize=(14, 6),dpi=160)

        # Plot 1: STD values
        scatter1 = axs[0].scatter(
            X, Y, 
            c=STD, 
            s=np.array(STD)*4, 
            cmap='plasma', 
            alpha=0.8, 
            edgecolors='k'
        )
        axs[0].set_title("Baseline RMS vs SiPM Positions")
        axs[0].set_xlabel("X [mm]")
        axs[0].set_ylabel("Y [mm]")
        plt.colorbar(scatter1, ax=axs[0], label="Log Scale STD")

        # Plot 2: Max values
        scatter2 = axs[1].scatter(X, Y, c=max_values, s=np.array(max_values)/10, cmap='plasma', alpha=0.8, edgecolors='k')
        axs[1].set_title("Max Amp vs SiPM Positions")
        axs[1].set_xlabel("X [mm]")
        axs[1].set_ylabel("Y [mm]")
        plt.colorbar(scatter2, ax=axs[1], label="Max Value")

        # Adjust layout and display
        plt.tight_layout()
        plt.show()

### Reco Irene Widgets

In [16]:
# Create Reco button
Reco_Irene_Button = widgets.Button(
    description='Setup Irene Reco',
    button_style='danger',  # 'primary' for a blue button
    layout=widgets.Layout(width='150px', margin='5px'),
    disabled=True
)

# Example variables that are constants
pes = 'pes'
adc = 'adc'
mus = 'mus'

# Define the editable variables
unshown_var= {
    "files_in": "$ICDIR/database/test_data/electrons_40keV_z25_RWF.h5",
    "file_out": "/tmp/electrons_40keV_z25_PMP.h5",
    "compression": "ZLIB4",
    "run_number": 14489,
    "print_mod": 1,
    "event_range": "34, 34+1",
    "detector_db": "next100"
}

variables = {
    "n_baseline": 62400,
    "n_maw": 100,
    "thr_maw": 3,  # Editable value
    "thr_csum_s1": 0.1,  # Editable value
    "thr_csum_s2": 0.1,  # Editable value
    "thr_sipm": 1.0,  # Editable value
    "thr_sipm_type": "common",
    "s1_tmin": 0,
    "s1_tmax": 1580,
    "s1_stride": 1,
    "s1_lmin": 8,
    "s1_lmax": 400,
    "s1_rebin_stride": 1,
    "s2_tmin": 1580,
    "s2_tmax": 2600,
    "s2_stride": 20,
    "s2_lmin": 400,
    "s2_lmax": 100000,
    "s2_rebin_stride": 80,
    "thr_sipm_s2": 50,  # Editable value
    "pmt_samp_wid": 25,  # Editable value
    "sipm_samp_wid": 1   # Editable value
}

units = {
    "thr_maw": "* adc",
    "thr_csum_s1": "* pes",
    "thr_csum_s2": "* pes",
    "thr_sipm": "* pes",
    "s1_tmin": "* mus",
    "s1_tmax": "* mus",
    "s2_tmin": "* mus",
    "s2_tmax": "* mus",
    "thr_sipm_s2": "* pes",
    "pmt_samp_wid": "* ns",
    "sipm_samp_wid": "* mus"
}

# Create the widgets
widgets_dict = {}

for key, value in variables.items():
    if isinstance(value, (int)):  # Only make editable for numerical values
        widgets_dict[key] = widgets.IntText(
            value=value,
            description=key + ":",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
    elif isinstance(value, float):  # Only make editable for numerical values
        widgets_dict[key] = widgets.FloatText(
            value=value,
            description=key + ":",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
    else:  # For string variables, we use Text widgets
        widgets_dict[key] = widgets.Text(
            value=value,
            description=key + ":",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )

columns = 4

# Split widgets into rows with the specified number of columns
widget_list = list(widgets_dict.values())
reco_params_raws = [
    widgets.HBox(widget_list[i:i + columns], layout=widgets.Layout(justify_content='space-between'))
    for i in range(0, len(widget_list), columns)
]

# Create Launch Reco button
Reset_Params_Default = widgets.Button(
    description='Reset Default',
    button_style='',  # No predefined style (gray button)
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px', margin='5px'),
    disabled=False
)

# Create Launch Reco button
Launch_Reco_Button = widgets.Button(
    description='Launch Reco',
    button_style='danger',  # 'primary' for a blue button
    layout=widgets.Layout(width='150px', margin='5px'),
    disabled=False
)

### Reco Irene Functions

In [17]:
def display_reco_ui(b):
    ui2.layout.display = 'block'
    
    unshown_var["run_number"]=int(run_number)
    unshown_var["event_range"]= (local_ev_number, local_ev_number+1)
    
def hide_display_reco_ui():
    ui2.layout.display = 'none'
    ui3.layout.display = 'none'
    
def reset_defaults_reco_par(b):
    for key, widget in widgets_dict.items():
        widget.value = variables[key]

def make_temp_file():
    # Open the file in write mode
    with open(config_file_path, "w") as config_file:
        # Write unshown_var contents
        for key, value in unshown_var.items():
            # Add quotes around string values
            if isinstance(value, str):
                config_file.write(f'{key} = "{value}"\n')
            else:
                config_file.write(f"{key} = {value}\n")

        for key, widget in widgets_dict.items():
            # Strip ":" from the description and write the value
            clean_key = widget.description.rstrip(":")

            # Check if the widget value is a string to add quotes
            if isinstance(widget.value, str):
                value_str = f'"{widget.value}"'
                if clean_key == "thr_sipm_type":  # Special case for thr_sipm_type
                    value_str = widget.value.strip('"')
            else:
                value_str = str(widget.value)
            
            # Check if units for the clean_key exist
            if clean_key in units:
                # If units exist, append the unit to the value
                config_file.write(f'{clean_key} = {value_str} {units[clean_key]}\n')
            else:
                # If no units, just write the value
                config_file.write(f'{clean_key} = {value_str}\n')

def deactivate_button_IreneDisplay():
    NextPMTS1_button.disabled=True
    PrevPMTS1_button.disabled=True
    NextPMTS2_button.disabled=True
    PrevPMTS2_button.disabled=True
    NextSiPMS2_button.disabled=True
    PrevSiPMS2_button.disabled=True 
                
def launch_reco_irene(b):
    ui3.layout.display = 'none'
    output_widget2.clear_output()
    
    make_temp_file()
    if os.path.exists('/tmp/tempout_irene.h5'):
        os.system("rm /tmp/tempout_irene.h5")
    cmd = f"city irene {config_file_path} -i {filename} -o /tmp/tempout_irene.h5"    

    os.system(cmd+"> /dev/null 2>&1")

    output_widget2.clear_output()
    
    global f_Irene 
    f_Irene = tb.open_file("/tmp/tempout_irene.h5", 'r')
    
    global df_S1_all
    df_S1_all = pd.DataFrame(f_Irene.get_node('/PMAPS/S1')[:])
    
    global df_S2_all
    df_S2_all = pd.DataFrame(f_Irene.get_node('/PMAPS/S2')[:])
    
    global df_S1_sing_PMT
    df_S1_sing_PMT = pd.DataFrame(f_Irene.get_node('/PMAPS/S1Pmt')[:])
    
    global df_S2_sing_PMT
    df_S2_sing_PMT = pd.DataFrame(f_Irene.get_node('/PMAPS/S2Pmt')[:])
    
    global df_S2_sing_SiPM
    df_S2_sing_SiPM = pd.DataFrame(f_Irene.get_node('/PMAPS/S2Si')[:])
    
    unique_SiPM_list=np.sort(df_S2_sing_SiPM['nsipm'].unique())
    #print(len(unique_SiPM_list))
    S2_SiPM_input.disabled = False
    
    if(len(unique_SiPM_list)>0):
        S2_SiPM_input.options=unique_SiPM_list
        S2_SiPM_input.value=unique_SiPM_list[0]
        S2_SiPM_input.disabled = False
        S2_SiPM_button.disabled = False
    else:
        S2_SiPM_input.options=[-1]
        S2_SiPM_input.value=-1
        S2_SiPM_input.disabled = True
        S2_SiPM_button.disabled = True
    
    deactivate_button_IreneDisplay()
    ui3.layout.display = 'block'

### Irene Display widgets

In [18]:
# Define buttons and input fields
S1_Sum_PMT_button = widgets.Button(
    description="PMAPS S1 Sum PMT",
    button_style='primary',
    layout=widgets.Layout(width='150px', margin='5px')
)

checkbox_logscale_S1_Sum_PMT = widgets.Checkbox(
    value=False,  # Default value
    description='log/lin',  # Label for the checkbox
    indent=False,  # Removes the default indentation
    layout=widgets.Layout(width='130px')
)

S2_Sum_PMT_button = widgets.Button(
    description="PMAPS S2 Sum PMT",
    button_style='primary',
    layout=widgets.Layout(width='150px', margin='5px')
)

checkbox_logscale_S2_Sum_PMT = widgets.Checkbox(
    value=False,  # Default value
    description='log/lin',  # Label for the checkbox
    indent=False,  # Removes the default indentation
    layout=widgets.Layout(width='130px')
)

S1_PMT_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=59,
    description='PMT:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='120px')
)

checkbox_logscale_S1_PMT = widgets.Checkbox(
    value=False,  # Default value
    description='log/lin',  # Label for the checkbox
    indent=False,  # Removes the default indentation
    layout=widgets.Layout(width='130px')
)

S1_PMT_button = widgets.Button(
    description="PMAPS S1 PMT",
    button_style='success',
    layout=widgets.Layout(width='120px', margin='5px')
)

S2_PMT_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=59,
    description='PMT:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='120px')
)

checkbox_logscale_S2_PMT = widgets.Checkbox(
    value=False,  # Default value
    description='log/lin',  # Label for the checkbox
    indent=False,  # Removes the default indentation
    layout=widgets.Layout(width='130px')
)

S2_PMT_button = widgets.Button(
    description="PMAPS S2 PMT",
    button_style='success',
    layout=widgets.Layout(width='120px', margin='5px')
)

S2_SiPM_input = widgets.SelectionSlider(
    options=[1, 2, 2, 3],
    value=2,
    description='N SiPM',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

S2_SiPM_button = widgets.Button(
    description="PMAPS S2 SiPM",
    button_style='success',
    layout=widgets.Layout(width='120px', margin='5px')
)

# Buttons for S1_PMT_input
NextPMTS1_button = widgets.Button(
    description='>>',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Smaller button width
)

PrevPMTS1_button = widgets.Button(
    description='<<',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Smaller button width
)

# Buttons for S2_PMT_input
NextPMTS2_button = widgets.Button(
    description='>>',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Smaller button width
)

PrevPMTS2_button = widgets.Button(
    description='<<',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Smaller button width
)

# Buttons for S2_SiPM_input
NextSiPMS2_button = widgets.Button(
    description='>>',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Smaller button width
)

PrevSiPMS2_button = widgets.Button(
    description='<<',
    button_style='success',
    style={'description_width': 'initial'},
    disabled=True,
    layout=widgets.Layout(width='40px')  # Smaller button width
)

# Create Output widget
output_widget2 = widgets.Output()

### Irene reco display functions

In [19]:
# Updated function to display the plot in output_widget2
def display_sum_PMT_S1(b):
    
    output_widget2.clear_output()
    # Calculate the average time for each peak
    average_times = df_S1_all.groupby('peak')['time'].mean()

    # Use output_widget2 for displaying the plot
    with output_widget2:
        plt.figure(figsize=(12, 4),dpi=160)
        plt.plot(df_S1_all['time'] / 1000, df_S1_all['ene'], marker='.', linestyle='', color='b', label='Energy vs Time')

        # Add average time markers
        plt.scatter(average_times / 1000, [df_S1_all['ene'].max() + 500] * len(average_times), 
                    color='red', marker='1', label='Avg Time per Peak', zorder=5)

        # Add labels, title, and grid
        plt.xlabel('Time ($\mu$s)')
        plt.ylabel('Energy')
        plt.title('Energy vs Time with Average Time per Peak')
        plt.grid(True)
        if(checkbox_logscale_S1_Sum_PMT.value):
            plt.yscale('log')
        plt.legend()
        plt.show()

# Updated function to display the plot in output_widget2
def display_sum_PMT_S2(b):
    output_widget2.clear_output()
    # Calculate the average time for each peak
    average_times = df_S2_all.groupby('peak')['time'].mean()

    # Use output_widget2 for displaying the plot
    with output_widget2:
        plt.figure(figsize=(12, 4), dpi=160)
        plt.plot(df_S2_all['time'] / 1000, df_S2_all['ene'], marker='.', linestyle='', color='b', label='Energy vs Time')

        # Add average time markers
        plt.scatter(average_times / 1000, [df_S2_all['ene'].max() + 500] * len(average_times), 
                    color='red', marker='1', label='Avg Time per Peak', zorder=5)

        # Add labels, title, and grid
        plt.xlabel('Time ($\mu$s)')
        plt.ylabel('Energy')
        plt.title('Energy vs Time with Average Time per Peak')
        plt.grid(True)
        if(checkbox_logscale_S2_Sum_PMT.value):
            plt.yscale('log')
        plt.legend()
        plt.show()
        
# Updated function to display the plot in output_widget2
def display_single_PMT_S1(b):
    output_widget2.clear_output()
    PrevPMTS1_button.disabled = False
    NextPMTS1_button.disabled = False

    # Get PMT positions (X, Y)
    PMTPos = DataPMT('next100', 0).filter('XY')
    X, Y = PMTPos["X"], PMTPos["Y"]

    # Current PMT index
    npmt = S1_PMT_input.value

    # Extract time and energy data
    time = df_S1_all['time']
    ene = df_S1_sing_PMT[(df_S1_sing_PMT['npmt'] == npmt)]['ene']

    with output_widget2:
        # Create a figure with two subplots
        fig, axes = plt.subplots(1, 2, figsize=(16, 4), dpi=160, gridspec_kw={'width_ratios': [1, 3]})

        # Left subplot: X vs Y with a yellow dot
        ax1 = axes[0]
        ax1.scatter(X, Y, color='gray', alpha=0.7, label='PMT positions')
        ax1.scatter(X[npmt], Y[npmt], color='yellow', s=100, label=f'PMT {npmt}')
        for i in range(len(X)):
            ax1.annotate(str(i), (X[i], Y[i]), textcoords="offset points", xytext=(8, 0), ha='center', fontsize=8, color='red')
        ax1.set_xlabel('X Position')
        ax1.set_ylabel('Y Position')
        ax1.set_title('PMT Map')

        # Right subplot: Energy vs Time
        ax2 = axes[1]
        ax2.plot(time, ene, marker='.', linestyle='', color='b', label='Energy vs Time')
        ax2.set_xlabel('Time ($\mu$s)')
        ax2.set_ylabel('Energy')
        ax2.set_title('Energy vs Time')
        if checkbox_logscale_S1_PMT.value:
            ax2.set_yscale('log')

        # Adjust layout and show the plots
        plt.tight_layout()
        plt.show()
        
def NextPMTS1Fun(b):
    value=S1_PMT_input.value
    if(value<num_pmts-1):
        S1_PMT_input.value=value+1
        display_single_PMT_S1(None)
    
def PrevPMTS1Fun(b):
    value=S1_PMT_input.value
    if(value>0):
        S1_PMT_input.value=value-1
        display_single_PMT_S1(None)

        
# Updated function to display the plot in output_widget2
def display_single_PMT_S2(b):
    output_widget2.clear_output()
    PrevPMTS2_button.disabled = False
    NextPMTS2_button.disabled = False

    # Get PMT positions (X, Y)
    PMTPos = DataPMT('next100', 0).filter('XY')
    X, Y = PMTPos["X"], PMTPos["Y"]

    npmt = S2_PMT_input.value

    time = df_S2_all['time']
    ene = df_S2_sing_PMT[(df_S2_sing_PMT['npmt'] == npmt)]['ene']

    # Use output_widget2 for displaying the plot
    with output_widget2:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4), dpi=160, gridspec_kw={'width_ratios': [1, 3]})

        # Left subplot: PMT positions
        ax1.scatter(X, Y, color='gray', label='PMT Positions')
        ax1.scatter(X[npmt], Y[npmt], color='yellow', label=f'PMT {npmt}', s=100)
        for i in range(len(X)):
            ax1.annotate(str(i), (X[i], Y[i]), textcoords="offset points", xytext=(8, 0), ha='center', fontsize=8, color='red')
        ax1.set_xlabel('X Position')
        ax1.set_ylabel('Y Position')
        ax1.set_title('PMT Positions')

        # Right subplot: Energy vs Time
        ax2.plot(time, ene, marker='.', linestyle='', color='b', label='Energy vs Time')
        ax2.set_xlabel('Time ($\mu$s)')
        ax2.set_ylabel('Energy')
        ax2.set_title('Energy vs Time with Average Time per Peak')
        ax2.grid(True)
        if checkbox_logscale_S2_PMT.value:
            ax2.set_yscale('log')

        # Show the plots
        plt.tight_layout()
        plt.show()


        
def NextPMTS2Fun(b):
    value=S2_PMT_input.value
    if(value<num_pmts-1):
        S2_PMT_input.value=value+1
        display_single_PMT_S2(None)
    
def PrevPMTS2Fun(b):
    value=S2_PMT_input.value
    if(value>0):
        S2_PMT_input.value=value-1
        display_single_PMT_S2(None)

        
# Updated function to display the plot in output_widget2
def display_single_SiPM_S2(b):
    output_widget2.clear_output()
    PrevSiPMS2_button.disabled=False
    NextSiPMS2_button.disabled=False
    
    # Extract SiPM positions
    SiPMpos = DataSiPM('next100', 0).filter('XY')

    # Extract X and Y positions
    X, Y = SiPMpos["X"], SiPMpos["Y"]

    # Get the selected SiPM value
    a_SiPM = S2_SiPM_input.value

    # Extract peaks associated with the selected SiPM
    peak_list_inSiPM = df_S2_sing_SiPM[df_S2_sing_SiPM['nsipm'] == a_SiPM]['peak'].unique()

    # Extract time and energy data
    time = df_S2_all[df_S2_all['peak'].isin(peak_list_inSiPM)]['time']
    ene = df_S2_sing_SiPM[df_S2_sing_SiPM['nsipm'] == a_SiPM]['ene']

    # Use output_widget2 for displaying the plots
    with output_widget2:
        plt.figure(figsize=(20, 6), dpi=160)

        # Define a grid layout with 2 columns
        gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3])  # Narrower X vs Y plot (1) and wider Energy vs Time plot (3)

        # Subplot 1: X vs Y (on the left)
        plt.subplot(gs[0])
        plt.scatter(X, Y, color='blue', s=20, alpha=0.5, label='SiPMs')
        plt.scatter(X[a_SiPM], Y[a_SiPM], color='yellow', s=80, label=f'Selected SiPM {a_SiPM}')
        plt.xlabel('X (mm)')
        plt.ylabel('Y (mm)')
        plt.title('SiPM Positions')
        plt.grid(True)

        # Subplot 2: Time vs Energy (on the right)
        plt.subplot(gs[1])
        plt.plot(time, ene, marker='.', linestyle='-', color='b', label='Energy vs Time')
        plt.xlabel('Time ($\mu$s)')
        plt.ylabel('Energy')
        plt.title('Energy vs Time')
        plt.grid(True)

        # Show the plots
        plt.tight_layout()
        plt.show()
    
        
def NextSiPMS2Fun(b):
    options = S2_SiPM_input.options
    current_val = S2_SiPM_input.value
    
    index = options.index(current_val)
    
    if index + 1 < len(options):
        S2_SiPM_input.value = options[index + 1]
        
    display_single_SiPM_S2(None)

def PrevSiPMS2Fun(b):
    options = S2_SiPM_input.options
    current_val = S2_SiPM_input.value
    
    index = options.index(current_val)
    
    if index - 1 >= 0:
        S2_SiPM_input.value = options[index - 1]
        
    display_single_SiPM_S2(None)
    

### Actions for buttons

In [29]:
# Attach events
check_run_button.on_click(check_folder_exists)

event_selection_button.on_click(select_event)
NextEvent_button.on_click(set_next_event)
PrevEvent_button.on_click(set_prev_event)


set_Ndef_button.on_click(set_N_default)
plot_pmt_button.on_click(plot_pmt_wfs)
NextPMT_button.on_click(plot_next_pmt)
PrevPMT_button.on_click(plot_prev_pmt)

SiPM_plot_button.on_click(plot_sipm_wf)
NextSiPM_button.on_click(plot_next_sipm)
PrevSiPM_button.on_click(plot_prev_sipm)

PMT_Stat_Plot.on_click(plotPMT_stat)
SiPM_Stat_Plot.on_click(plotSiPM_stat)

Reco_Irene_Button.on_click(display_reco_ui)
Reset_Params_Default.on_click(reset_defaults_reco_par)
Launch_Reco_Button.on_click(launch_reco_irene)

S1_Sum_PMT_button.on_click(display_sum_PMT_S1)
S2_Sum_PMT_button.on_click(display_sum_PMT_S2)

S1_PMT_button.on_click(display_single_PMT_S1)
PrevPMTS1_button.on_click(PrevPMTS1Fun)
NextPMTS1_button.on_click(NextPMTS1Fun)

S2_PMT_button.on_click(display_single_PMT_S2)
PrevPMTS2_button.on_click(PrevPMTS2Fun)
NextPMTS2_button.on_click(NextPMTS2Fun)

S2_SiPM_button.on_click(display_single_SiPM_S2)
PrevSiPMS2_button.on_click(PrevSiPMS2Fun)
NextSiPMS2_button.on_click(NextSiPMS2Fun)

### UI Definition

In [21]:
################ CREATING FIRST UI

run_box = VBox([run_number_input_form, check_run_button, output], layout=widgets.Layout(margin='10px', align_items='flex-start'))
event_button_with_bar=HBox([event_selection_button,PrevEvent_button,NextEvent_button ,progress_bar_load_ev])
event_sel_box = VBox([event_select_slider, event_button_with_bar], layout=widgets.Layout(margin='10px', align_items='flex-start'))

#Define PMT box
pmt_box = VBox(
    [   HBox([pmt_number_input,checkbox_logscale_PMTrw])   , deconv_par_input, HBox([plot_pmt_button, set_Ndef_button, PrevPMT_button, NextPMT_button])], 
    layout=widgets.Layout(margin='10px', align_items='flex-start')
)

#Define SiPM box
plot_sipm_box = HBox([SiPM_plot_button,PrevSiPM_button,NextSiPM_button])
sipm_box = VBox([ SiPM_number_input , plot_sipm_box], layout=widgets.Layout(margin='10px', align_items='flex-start'))


#Define a statistics box
Stastistics_box = HBox([PMT_Stat_Plot,SiPM_Stat_Plot])

# Create a spacer widget for more spacing between S2_PMT_box and S2_SiPM_box
spacer = widgets.Box(layout=widgets.Layout(width='30px'))

# Create top row of widget
top_row = HBox([run_box,spacer,event_sel_box, spacer,pmt_box], layout=widgets.Layout(margin='10px', align_items='flex-start'))
# Create second row of widget
second_row = HBox( [sipm_box,spacer,Stastistics_box,spacer,spacer,Reco_Irene_Button], layout=widgets.Layout(margin='10px', align_items='flex-start')) 

################ FIRST UI TO DISPLAY
ui = VBox([top_row, second_row ,plot_output], layout=widgets.Layout(margin='10px', justify_content='flex-start'))

# Create an HBox for the buttons, aligned at the center
buttons_box = widgets.HBox(
    [Reset_Params_Default, Launch_Reco_Button],
    layout=widgets.Layout(justify_content='center', spacing='10px', margin='10px')
)

######### SECOND UI TO DISPLAY
ui2 = widgets.VBox(
    reco_params_raws + [buttons_box],  # Add buttons_box below reco_params_raws
    layout=widgets.Layout(spacing='10px', justify_content='flex-start')  # Optional spacing/layout tweaks
)



#######STARTING 3rd UI
# Arrange the widgets
left_column = widgets.VBox(
    [      HBox([S1_Sum_PMT_button,checkbox_logscale_S1_Sum_PMT]),  HBox([S2_Sum_PMT_button,checkbox_logscale_S2_Sum_PMT])], 
    layout=widgets.Layout(align_items='center', spacing='20px')  # Added spacing
)

# Layout for S2_PMT_input, S2_PMT_button, and navigation buttons
S1_PMT_box = widgets.VBox(
    [
        HBox([S1_PMT_input,checkbox_logscale_S1_PMT]),    
        widgets.HBox(
            [ S1_PMT_button, PrevPMTS1_button, NextPMTS1_button], 
            layout=widgets.Layout(align_items='center', justify_content='flex-start', spacing='10px')  # Adjust alignment
        )
    ],
    layout=widgets.Layout(align_items='flex-start', spacing='10px')  # Ensure alignment
)


# Layout for S2_PMT_input, S2_PMT_button, and navigation buttons
S2_PMT_box = widgets.VBox(
    [
        HBox([S2_PMT_input,checkbox_logscale_S2_PMT]),    
        widgets.HBox(
            [ S2_PMT_button, PrevPMTS2_button, NextPMTS2_button], 
            layout=widgets.Layout(align_items='center', justify_content='flex-start', spacing='10px')  # Adjust alignment
        )
    ],
    layout=widgets.Layout(align_items='flex-start', spacing='10px')  # Ensure alignment
)

# Layout for S2_SiPM_input, S2_SiPM_button, and navigation buttons
S2_SiPM_box = widgets.VBox(
    [
        S2_SiPM_input,      
        widgets.HBox(
            [ S2_SiPM_button, PrevSiPMS2_button, NextSiPMS2_button], 
            layout=widgets.Layout(align_items='center', justify_content='flex-start', spacing='10px')  # Adjust alignment
        )
    ],
    layout=widgets.Layout(align_items='flex-start', spacing='10px')  # Ensure alignment
)

# Create a spacer widget for more spacing between S2_PMT_box and S2_SiPM_box
spacer2 = widgets.Box(layout=widgets.Layout(width='10px'))

# Combine S2_PMT_box, spacer, and S2_SiPM_box
right_column = widgets.HBox(
    [S1_PMT_box,spacer,S2_PMT_box, spacer2, S2_SiPM_box],
    layout=widgets.Layout(justify_content='center', align_items='center')
)


# Combine left and right columns
comands_ui3 = widgets.HBox(
    [left_column, spacer2, right_column], 
    layout=widgets.Layout(justify_content='space-around', align_items='center', spacing='200px')  # Adjust spacing and alignment
)

########## THIRD UI TO DISPLAY
ui3 = widgets.VBox([comands_ui3,output_widget2])

display(ui)
display(ui2)
ui2.layout.display = 'none'
display(ui3)
ui3.layout.display = 'none'

#N 14550 - 83 for test